In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

C:\Users\pwrxn\miniconda3\envs\pytorch\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train= pd.read_csv('train.csv')
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [3]:
# Load datasets
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays_events = pd.read_csv('holidays_events.csv')
# transactions = pd.read_csv('transactions.csv')

# Convert date columns to datetime type
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
# transactions['date'] = pd.to_datetime(transactions['date'])

# Merge store metadata with train and test datasets
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')

# Merge oil prices with train and test datasets
train = train.merge(oil, on='date', how='left')
test = test.merge(oil, on='date', how='left')

# Merge holidays and events data
# Assuming you want to include this data for each store and each date
train = train.merge(holidays_events, on='date', how='left')
test = test.merge(holidays_events, on='date', how='left')

# Merge transactions with trian and test datasets
# train = train.merge(transactions, on='date', how='left')
# test = test.merge(transactions, on='date', how='left')

# Inspect the merged train dataset
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
 6   city         object        
 7   state        object        
 8   type_x       object        
 9   cluster      int64         
 10  dcoilwtico   float64       
 11  type_y       object        
 12  locale       object        
 13  locale_name  object        
 14  description  object        
 15  transferred  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(9)
memory usage: 372.8+ MB


In [5]:
train.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
dcoilwtico      955152
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dtype: int64

In [6]:
# Forward fill or interpolate missing oil prices
train['dcoilwtico'] = train['dcoilwtico'].bfill().ffill()  # or.interpolate()

# Handling holidays/events columns
for col in ['type_y', 'locale', 'locale_name', 'description', 'transferred']:
    train[col] = train[col].fillna('None')

# Creating a binary feature for holiday/event
train['is_holiday_event'] = train['type_y'] != 'None'

In [7]:
train.isna().sum()

id                  0
date                0
store_nbr           0
family              0
sales               0
onpromotion         0
city                0
state               0
type_x              0
cluster             0
dcoilwtico          0
type_y              0
locale              0
locale_name         0
description         0
transferred         0
is_holiday_event    0
dtype: int64

In [8]:
train.family.value_counts()

family
AUTOMOTIVE                    92556
HOME APPLIANCES               92556
SCHOOL AND OFFICE SUPPLIES    92556
PRODUCE                       92556
PREPARED FOODS                92556
POULTRY                       92556
PLAYERS AND ELECTRONICS       92556
PET SUPPLIES                  92556
PERSONAL CARE                 92556
MEATS                         92556
MAGAZINES                     92556
LIQUOR,WINE,BEER              92556
LINGERIE                      92556
LAWN AND GARDEN               92556
LADIESWEAR                    92556
HOME CARE                     92556
HOME AND KITCHEN II           92556
BABY CARE                     92556
HOME AND KITCHEN I            92556
HARDWARE                      92556
GROCERY II                    92556
GROCERY I                     92556
FROZEN FOODS                  92556
EGGS                          92556
DELI                          92556
DAIRY                         92556
CLEANING                      92556
CELEBRATION          

In [9]:
train.city.value_counts()

city
Quito            1018116
Guayaquil         452496
Santo Domingo     169686
Cuenca            169686
Latacunga         113124
Manta             113124
Machala           113124
Ambato            113124
Quevedo            56562
Esmeraldas         56562
Loja               56562
Libertad           56562
Playas             56562
Daule              56562
Babahoyo           56562
Cayambe            56562
Salinas            56562
Puyo               56562
Guaranda           56562
Ibarra             56562
Riobamba           56562
El Carmen          56562
Name: count, dtype: int64

In [10]:
train.type_x.value_counts()

type_x
D    1018116
C     848430
A     509058
B     452496
E     226248
Name: count, dtype: int64

In [11]:
train.state.value_counts()

state
Pichincha                         1074678
Guayas                             622182
Santo Domingo de los Tsachilas     169686
Azuay                              169686
Manabi                             169686
Cotopaxi                           113124
Tungurahua                         113124
Los Rios                           113124
El Oro                             113124
Chimborazo                          56562
Imbabura                            56562
Bolivar                             56562
Pastaza                             56562
Santa Elena                         56562
Loja                                56562
Esmeraldas                          56562
Name: count, dtype: int64

In [12]:
# One-hot encoding for 'type_x'
train = pd.get_dummies(train, columns=['type_x'], drop_first=True)

# Frequency encoding for 'family'
family_freq = train['family'].value_counts() / len(train)
train['family'] = train['family'].map(family_freq)

# Frequency encoding for 'city'
city_freq = train['city'].value_counts() / len(train)
train['city'] = train['city'].map(city_freq)

# Frequency encoding for 'state'
state_freq = train['state'].value_counts() / len(train)
train['state'] = train['state'].map(state_freq)

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 20 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                int64         
 1   date              datetime64[ns]
 2   store_nbr         int64         
 3   family            float64       
 4   sales             float64       
 5   onpromotion       int64         
 6   city              float64       
 7   state             float64       
 8   cluster           int64         
 9   dcoilwtico        float64       
 10  type_y            object        
 11  locale            object        
 12  locale_name       object        
 13  description       object        
 14  transferred       object        
 15  is_holiday_event  bool          
 16  type_x_B          bool          
 17  type_x_C          bool          
 18  type_x_D          bool          
 19  type_x_E          bool          
dtypes: bool(5), datetime64[ns](1), float64(5), int

In [14]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,is_holiday_event,type_x_B,type_x_C,type_x_D,type_x_E
0,0,2013-01-01,1,0.030303,0.0,0,0.333333,0.351852,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,True,False,False,True,False
1,1,2013-01-01,1,0.030303,0.0,0,0.333333,0.351852,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,True,False,False,True,False
2,2,2013-01-01,1,0.030303,0.0,0,0.333333,0.351852,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,True,False,False,True,False
3,3,2013-01-01,1,0.030303,0.0,0,0.333333,0.351852,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,True,False,False,True,False
4,4,2013-01-01,1,0.030303,0.0,0,0.333333,0.351852,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,True,False,False,True,False


In [15]:
train.type_y.value_counts()

type_y
None          2551824
Holiday        304722
Event           99792
Additional      67716
Transfer        16038
Work Day         8910
Bridge           5346
Name: count, dtype: int64

In [16]:
train.locale.value_counts()

locale
None        2551824
National     261954
Local        208494
Regional      32076
Name: count, dtype: int64

In [17]:
train.locale_name.value_counts()

locale_name
None                              2551824
Ecuador                            261954
Guayaquil                           19602
Riobamba                            17820
Guaranda                            16038
Latacunga                           16038
Quito                               14256
Ambato                              14256
Cuenca                              10692
Esmeraldas                           8910
Cayambe                              8910
Santo Domingo                        8910
El Carmen                            8910
Machala                              8910
Imbabura                             8910
Puyo                                 8910
Libertad                             8910
Cotopaxi                             8910
Manta                                8910
Ibarra                               7128
Quevedo                              7128
Santo Domingo de los Tsachilas       7128
Santa Elena                          7128
Loja                  

In [18]:
train.description.value_counts()

description
None                           2551824
Carnaval                         17820
Fundacion de Cuenca              10692
Primer dia del ano                8910
Cantonizacion de Guaranda         8910
                                ...   
Terremoto Manabi+6                1782
Terremoto Manabi+7                1782
Terremoto Manabi+8                1782
Terremoto Manabi+9                1782
Traslado Primer dia del ano       1782
Name: count, Length: 102, dtype: int64

In [19]:
train.transferred.value_counts()

transferred
None     2551824
False     486486
True       16038
Name: count, dtype: int64

In [20]:
# One-hot encoding
one_hot_columns = ['type_y', 'locale']
train = pd.get_dummies(train, columns=one_hot_columns, drop_first=True)

# Frequency encoding for 'locale_name'
locale_name_freq = train['locale_name'].value_counts() / len(train)
train['locale_name'] = train['locale_name'].map(locale_name_freq)

# Frequency encoding for 'description'
description_freq = train['description'].value_counts() / len(train)
train['description'] = train['description'].map(description_freq)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 27 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                int64         
 1   date              datetime64[ns]
 2   store_nbr         int64         
 3   family            float64       
 4   sales             float64       
 5   onpromotion       int64         
 6   city              float64       
 7   state             float64       
 8   cluster           int64         
 9   dcoilwtico        float64       
 10  locale_name       float64       
 11  description       float64       
 12  transferred       object        
 13  is_holiday_event  bool          
 14  type_x_B          bool          
 15  type_x_C          bool          
 16  type_x_D          bool          
 17  type_x_E          bool          
 18  type_y_Bridge     bool          
 19  type_y_Event      bool          
 20  type_y_Holiday    bool          
 21  type_y_N

In [22]:
def preprocessing(df):
    '''Transforms the given dataset into encoded one.'''
    
    # One-hot encoding for 'type_x'
    df = pd.get_dummies(df, columns=['type_x'], drop_first=True)

    # Frequency encoding for 'family'
    family_freq = df['family'].value_counts() / len(df)
    df['family'] = df['family'].map(family_freq)

    # Frequency encoding for 'city'
    city_freq = df['city'].value_counts() / len(df)
    df['city'] = df['city'].map(city_freq)

    # Frequency encoding for 'state'
    state_freq = df['state'].value_counts() / len(df)
    df['state'] = df['state'].map(state_freq)

    # Forward fill or interpolate missing oil prices
    df['dcoilwtico'] = df['dcoilwtico'].bfill().ffill()  # or df['dcoilwtico'].interpolate()

    # Handling holidays/events columns
    for col in ['type_y', 'locale', 'locale_name', 'description', 'transferred']:
        df[col] = df[col].fillna('None')

    # Creating a binary feature for holiday/event
    df['is_holiday_event'] = df['type_y'] != 'None'

    # One-hot encoding
    one_hot_columns = ['type_y', 'locale']
    df = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

    # Frequency encoding for 'locale_name'
    locale_name_freq = df['locale_name'].value_counts() / len(df)
    df['locale_name'] = df['locale_name'].map(locale_name_freq)

    # Frequency encoding for 'description'
    description_freq = df['description'].value_counts() / len(df)
    df['description'] = df['description'].map(description_freq)

    # Label encoding for 'transferred'
    df['transferred'] = df['transferred'].map({'None': -1, 'False': 0, 'True': 1})  # -1 for 'None'

    return df

In [23]:
train.transferred.value_counts()

transferred
None     2551824
False     486486
True       16038
Name: count, dtype: int64

In [24]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Replace 'None' with numpy NaN for proper encoding
train['transferred'].replace({'None': None}, inplace=True)

# Fit the label encoder and transform the 'transferred' column
train['transferred'] = label_encoder.fit_transform(train['transferred'].astype(str))

# If you need to transform back to the original labels later
# labels = label_encoder.inverse_transform(train['transferred'])

train['transferred'], _ = pd.factorize(train['transferred'], sort=True)

# This will encode 'None', 'False', 'True' to 0, 1, 2 respectively
# If you want 'None' to be -1, you can simply subtract 1
train['transferred'] = train['transferred'] - 1

C:\Users\pwrxn\AppData\Local\Temp\ipykernel_22484\180067380.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['transferred'].replace({'None': None}, inplace=True)


In [25]:
train.transferred.value_counts()

transferred
 0    2551824
-1     486486
 1      16038
Name: count, dtype: int64

In [26]:
from sklearn.ensemble import RandomForestRegressor # import RandomForestRegressor

In [27]:
X = train.drop('sales', axis=1)
y = train['sales']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test

(              id       date  store_nbr    family  onpromotion      city  \
 580605    573477 2013-11-18          5  0.030303            0  0.055556   
 457845    450717 2013-09-10          6  0.030303            0  0.333333   
 2585113  2540563 2016-11-29         42  0.030303            8  0.055556   
 1839313  1814365 2015-10-18         17  0.030303            0  0.333333   
 2829206  2781397 2017-04-14          5  0.030303            9  0.055556   
 ...          ...        ...        ...       ...          ...       ...   
 1692743  1667795 2015-07-27         54  0.030303            0  0.018519   
 2356330  2313562 2016-07-25         23  0.030303            0  0.037037   
 2229084  2193444 2016-05-18         53  0.030303            3  0.037037   
 2768307  2721975 2017-03-12         33  0.030303           18  0.018519   
 2219110  2183470 2016-05-13         23  0.030303            1  0.037037   
 
             state  cluster  dcoilwtico  locale_name  ...  type_x_E  \
 580605   0.055

## Let's create a RandomForestRegressor model

In [29]:
# First ensure 'date' is a datetime dtype
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

# Now extract date features
X_train['year'] = X_train['date'].dt.year
X_train['month'] = X_train['date'].dt.month
X_train['day'] = X_train['date'].dt.day
X_train['weekday'] = X_train['date'].dt.weekday
# ...and similarly for X_test
X_test['year'] = X_test['date'].dt.year
X_test['month'] = X_test['date'].dt.month
X_test['day'] = X_test['date'].dt.day
X_test['weekday'] = X_test['date'].dt.weekday
# Now you can convert 'date' to timestamps if you need that representation
# Note: This step is optional and only necessary if you want to use timestamps as features
X_train['timestamp'] = X_train['date'].apply(lambda x: x.timestamp())
X_test['timestamp'] = X_test['date'].apply(lambda x: x.timestamp())

# Drop the original 'date' column if it's no longer needed
X_train.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)

In [30]:
from sklearn.ensemble import GradientBoostingRegressor
# Set the Model
gbr = GradientBoostingRegressor()

# Fit the model 
gbr.fit(X_train, y_train)

# Score the model
gbr_score = gbr.score(X_test, y_test)
gbr_score

0.4213338507219625

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

